# Code Section

Load all shared code for the analysis.


In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from io import StringIO

## Banco Central do Brasil - API séries históricas.

In [24]:
#URL base da API de séries históricas do Banco Central do Brasil.
BCB_SGS_API_URL = r'https://api.bcb.gov.br/dados'

# Código das séries históricas no SGS do BC. Os códigos podem ser acessados através da página do SGS do BC.
SGS_INPC: int = 188
SGS_IPCA: int = 433

# Recupera dados de série histórica do banco central, atrávés do código da série.
def get_sgs_data(serie):
  url = r'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(serie)
  df: pd.DataFrame = pd.read_json(url)
  return df


## IBGE - API SIDRA

### Parâmetros de consulta da API SIDRA

* Os parâmetros V e P permitem a seleção das variáveis e o período de tempo, respectivamente.
* Parâmetros iniciados por C são utilizados para selecionar classificações e suas categorias.<br> 
  As classificações disponíveis variam de acordo com a tabela, até um máximo de 6.
* Para especificação do território, pode-se utilizar o parâmetro  G  para identificar uma visão territorial predefinida, ou então um ou mais parâmetros do tipo N para indicar um nível territorial e suas unidades. Os tipos de parâmetro (G ou N) são excludentes entre si não podendo ser utilizados simultaneamente em uma mesma consulta.

-----

### Parâmetros de dimensão

#### **T** – para especificar a tabela de onde se deseja extrair os dados.

  O parâmetro T deve ser seguido pelo código numérico da tabela SIDRA. Este parâmetro é obrigatório, pois a tabela indicada servirá de base para todos os outros parâmetros da consulta.<br>
  Exemplo: /t/1612

-----

#### **P** – para especificar os períodos (meses, anos etc.)

  O parâmetro P é opcional e, caso não seja especificado, assumirá o valor default /p/last  (vide adiante).<br>
  O parâmetro P pode ser seguido pela constante all para selecionar todos os períodos disponíveis para a tabela.<br>
  Exemplo 1: /p/all

  O parâmetro P pode ser seguido pela constante first e um número, para indicar os primeiros períodos disponíveis para a tabela (períodos mais antigos). O número de períodos pode ser omitido quando se tratar de apenas um período.<br>
  Exemplo 2: /p/first 12<br>
  Exemplo 3: /p/first (equivalente a /p/first 1)<br>

  O parâmetro P pode ser seguido pela constante last e um número, para indicar os últimos períodos disponíveis para a tabela (períodos mais recentes). O número de períodos pode ser omitido quando se tratar de apenas um período.<br>
  Exemplo 4: /p/last 12<br>
  Exemplo 5: /p/last (equivalente a /p/last 1)<br>

  O parâmetro P pode especificar códigos de períodos de forma avulsa (separados por vírgula ","), em faixas (separados por traço "-"), ou de ambas as formas. Para mais detalhes sobre os códigos de períodos, consulte a nota adiante.<br>
  Exemplo 6: /p/2008,2009,2010 – especifica os anos 2008, 2009 e 2010.<br>
  Exemplo 7: /p/2008,2010-2014 – especifica o ano de 2008, e os anos no intervalo de 2010 a 2014.<br>
  Exemplo 8: /p/201101-201112,201204,201208 – especifica os meses de janeiro a dezembro de 2011, abril de 2012 e agosto de 2012.<br>

  > Nota<br>
  > O formato de um código de período é determinado pela periodicidade de liberação da tabela SIDRA consultada. Para tabelas de divulgação anual, o código possui 4 dígitos no formato AAAA, e indica o ano da ocorrência desejada. Para os demais casos, o código de período possui 6 dígitos no formato AAAASS, onde AAAA indica o ano e SS número de sequência da ocorrência que, conforme a periodicidade, pode representar um mês (01 a 12), um trimestre (01 a 04), um semestre (01 ou 02), entre outros.

-----

#### **V** – para especificar as variáveis<br>
  
  O parâmetro V é opcional e, caso não seja especificado, assumirá o valor default /p/allxp (vide adiante).<br>
  O parâmetro V pode ser seguido pela constante all para especificar todas as variáveis da tabela, inclusive as variáveis de percentual geradas automaticamente pelo Sidra.<br>
  Exemplo 1: /v/all
  
  O parâmetro V pode ser seguido pela constante allxp para especificar todas as variáveis da tabela, exceto as variáveis de percentual geradas automaticamente pelo Sidra.<br>
  Exemplo 2: /v/allxp

  O parâmetro V pode ser seguido por uma lista de códigos de variáveis (separados por vírgula ",").<br>
  Exemplo 3: /v/63,69 seleciona as variáveis "Variação mensal" e "Variação acumulada ao ano".<br>
  Exemplo 4: /v/109,1000109 – seleciona a variável "Área Plantada" e a variável calculada "Área plantada - percentual do total geral".

  > Nota<br>
  > Variáveis com códigos superiores a 1.000.000 identificam variáveis de percentual calculadas automaticamente. A disponibilidade de uma variável derivada deve ser consultada para cada tabela.

-----

#### **G** – para especificar uma visão territorial
  
  O parâmetro G deve ser seguido pelo código da visão territorial de interesse.<br>
  Exemplo: /g/44 – seleciona os valores para a visão territorial "Brasil e Grande Região".
  
  > Nota<br>
  > A disponibilidade de dados para uma determinada visão territorial deve ser consultada para cada tabela. Todas as unidades territorais da visão serão apresentadas na tabela de resultado, mesmo quando não houver disponibilidade. Nestes casos, o valor "..." será retornado. Quando utilizado o parâmetro G, não será possível utilizar o parâmetro Ni para especificar níveis territoriais avulsos.

-----

#### **Ni** – para especificar níveis territoriais e suas unidades
  O valor de Ni identifica o nível territorial (por exemplo N1 – Brasil, N2 – Grande Região, N3 – Unidade da Federação, N6 – Município, etc), e é seguido por uma seleção de unidades territoriais a serem consultadas. Este parâmetro pode ser utilizado mais de uma vez para especificar unidades territoriais em níveis distintos.<br>
  O parâmetro Ni pode ser seguido pela constante all para especificar todas as unidades territoriais deste nível disponíveis para a tabela.<br>
  Exemplo 1: /n3/all – especifica todas as unidades da federação.<br>
  Exemplo 2: /n6/all – especifica todos os municípios.<br>
  
  O parâmetro Ni pode ser seguido por uma lista de unidades territoriais especificadas por seus códigos IBGE (separados por vírgula ",").<br>
  Exemplo 3: /n3/33,35 – especifica as UFs Rio de Janeiro e São Paulo.<br>
  Exemplo 4: /n6/3304557,3550308 – especifica os municípios Rio de Janeiro e São Paulo.<br>
  
  O parâmetro Ni pode ser seguido pela expressão IN Nj antecedendo a lista de unidades territoriais, para que estas sejam tratadas como uma abrangência territorial. Em outras palavras, estaremos selecionando unidades territoriais do nível Ni, contidas em unidades territoriais de um nível superior Nj.<br>
  Exemplo 5: /n6/in n3 11,12 - especifica os municípios (N6) contidos nas Unidades da Federação (N3) Rondônia e Acre.<br>
  Exemplo 6: /n3/in n2 3,4 – especifica as Unidades da Federação (N3) contidas nas Grandes Regiões (N2) Sudeste e Sul.<br>
  
  O parâmetro Ni pode ser utilizado mais de uma vez caso sejam selecionadas unidades territoriais em níveis distintos.<br>
  Exemplo 7: /n1/1/n2/1/n3/in n2 1 – especifica Brasil (N1/1), a Grande Região Norte (N2/1) e as UFs contidas na Grandes Região Norte (N3/IN N2 1).
  
  > Notas<br>
  > A disponibilidade de dados para um determinado nível territorial deve ser consultada para cada tabela. A consulta, por padrão, não retorna unidades territoriais extintas. Caso seja necessário obter valores também para territórios extintos, tanto para consultas do tipo /all quanto por código (por exemplo as UFs Fernando de Noronha - 20 e Guanabara - 34), deve-se acrescentar na lista de parâmetros da consulta /u/y.<br>
  > Quando utilizado o parâmetro Ni, não será possível utilizar o parâmetro G para especificar uma visão territorial pré-definida.

-----

#### **Ci** – para especificar as classificações da tabela e suas categorias
  O valor de Ci identifica uma classificação da tabela (por exemplo C1 – Situação do domicílio, C2 – Sexo, C81 – Produtos da Lavoura Temporária, etc), e é seguido por uma seleção de categorias a serem consultadas.<br>
  O parâmetro Ci é opcional e, caso uma classificação não seja especificada, será assumida para ela a categoria que representa o total. Se esta categoria não existir na classificação, a consulta retornará o símbolo de que os valores não se aplicam (".." , 
  vide ao final).<br>
  O parâmetro Ci pode ser seguido pela constante all para especificar todas as categorias disponíveis para uma classificação, inclusive a categoria que representa o total.<br>
  Exemplo 1: /c81/all – especifica todos os produtos da lavoura temporária, incluindo o total.
  
  O parâmetro Ci pode ser seguido pela constante allxt para especificar todas as categorias disponíveis para uma classificação, exceto a categoria que representa o total.<br>
  Exemplo 2: /c81/allxt – especifica todos os produtos da lavoura temporária, sem incluir o total.
  
  O parâmetro Ci pode ser seguido por uma lista de categorias especificadas através de seus códigos, de forma individual (separados por vírgula ",") ou para compor uma soma (separadas por espaço " ").<br>
  Exemplo 3: /c81/2692,2702,2694 2695 – especifica os produtos da lavoura temporária arroz e feijão, e a soma de batata doce com batata inglesa.

In [25]:
# URL base da API do sistema SIDRA do IBGE.
SIDRA_BASE_URL = r'https://apisidra.ibge.gov.br/values'

# Parâmetros de consulta dos dados do INPC.
SIDRA_INPC_TABLEID: int = 1737
SIDRA_INPC_PARAMS = {
    'n1': 'all',
    'n3': 'all',
    'v': '63'}

SIDRA_IPCA_TABLEID = 1737
SIDRA_IPCA_PARAMS = {
    'n1': '1',
    'v': '63'
}

# Recupera dados das tabelas do sistema SIDRA do IBGE
def get_sidra_data(base_url, table_id, params):
  
  param_str:str = "/".join([f"{key}/{value}" for key, value in params.items()])
  url: str = f"{base_url}/t/{table_id}/{param_str}"

  try:
    response: requests.Response = requests.get(url)
    response: requests.Response = requests.get(url)
    response.raise_for_status()  # Lança uma exceção para códigos de erro HTTP
    data = response.json()
    df: pd.DataFrame = pd.read_json(data)
    return df
  except requests.exceptions.RequestException as e:
      print(f"Erro ao fazer a requisição: {e}")
  except (ValueError, KeyError) as e:
      print(f"Erro ao processar o JSON recebido: {e}")
  except Exception as e:
      print(f"Ocorreu um erro inesperado: {e}")

#Recupera o IPCA variação mensal dos últimos N meses
def get_ipca_variacao_mensal(ultimos_meses=12):
    """
    Busca a variação mensal do IPCA (Índice Geral) para o Brasil na API do IBGE (SIDRA)
    utilizando o formato JSON.

    :param ultimos_meses: O número de últimos meses a serem buscados.
    :return: Um DataFrame do Pandas com os dados ou None em caso de erro.
    """
    # Tabela 1737: IPCA - Variação mensal.
    # Variável 63: IPCA - Variação mensal.
    # Classificação C315/7169: Índice geral.
    # Nível Territorial N1/1: Brasil.
    # Período: /p/last%20{ultimos_meses} -> Últimos N meses.

    url = f"https://apisidra.ibge.gov.br/values/t/1737/n1/1/v/63/p/last%20{ultimos_meses}?formato=json"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança uma exceção para códigos de erro HTTP

        data = response.json()

        # A primeira linha da resposta contém os cabeçalhos
        headers = data[0]
        
        # Encontra as chaves para as colunas de mês e valor
        col_mes = next((key for key, value in headers.items() if value == 'Mês'), None)
        col_valor = next((key for key, value in headers.items() if 'Valor' in value), None)

        if not col_mes or not col_valor:
            print("Não foi possível encontrar as colunas de Mês e Valor na resposta da API.")
            print("Cabeçalhos recebidos:", headers)
            return None

        # As linhas de dados começam a partir do segundo item
        dados = data[1:]
        
        # Cria o DataFrame
        df = pd.DataFrame(dados)

        # Renomeia as colunas de interesse
        df.rename(columns={
            col_mes: 'Mes',
            col_valor: 'VariacaoMensal'
        }, inplace=True)
        
        # Converte a coluna de variação para tipo numérico
        df['VariacaoMensal'] = pd.to_numeric(df['VariacaoMensal'], errors='coerce')

        # Seleciona e reordena as colunas que queremos no resultado final
        df = df[['Mes', 'VariacaoMensal']]

        print(f"Variação mensal do IPCA (Brasil) - Últimos {ultimos_meses} meses (via JSON):")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição: {e}")
    except (ValueError, KeyError) as e:
        print(f"Erro ao processar o JSON recebido: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

    return None


## IBGE - API Dados Agregados

In [26]:
# URL base da API de dados agregados do IBGE.
agregados_base_url = r'https://servicodados.ibge.gov.br/api/v3/agregados'

# Data Section

## Preços e custos

Compreende as informações sobre as variações de preços dos bens e serviços produzidos na economia, abrangendo índices de preços ao consumidor, índices de preços ao produtor, e, ainda, comparações internacionais, visando medir as paridades de poder de compra e os níveis correspondentes de preço e volume do Produto Interno Bruto - PIB, entre outros aspectos. Compreende também as informações sobre custos e índices da construção civil.

### IPCA - Índice Nacional de Preços ao Consumidor Amplo

https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=o-que-e


O Índice Nacional de Preços ao Consumidor Amplo – IPCA que tem por objetivo medir a inflação de um conjunto de produtos e serviços comercializados no varejo, referentes ao consumo pessoal das famílias. Esta faixa de renda foi criada com o objetivo de garantir uma cobertura de 90% das famílias pertencentes às áreas urbanas de cobertura do Sistema Nacional de Índices de Preços ao Consumidor - SNIPC.

Esse índice de preços tem como unidade de coleta estabelecimentos comerciais e de prestação de serviços, concessionária de serviços públicos e intenet e sua coleta estende-se, em geral, do dia 01 a 30 do mês de referência.

Atualmente, a população-objetivo do IPCA abrange as famílias com rendimentos de 1 a 40 salários mínimos, qualquer que seja a fonte, residentes nas áreas urbanas das regiões de abrangência do SNIPC, as quais são: regiões metropolitanas de Belém, Fortaleza, Recife, Salvador, Belo Horizonte, Vitória, Rio de Janeiro, São Paulo, Curitiba, Porto Alegre, além do Distrito Federal e dos municípios de Goiânia, Campo Grande, Rio Branco, São Luís e Aracaju.

In [ ]:
SIDRA_BASE_URL: str = 'https://apisidra.ibge.gov.br/values'
SIDRA_INPC_TABLEID: int = 1736
SIDRA_IPCA_TABLEID: int = 1737


SIDRA_IPCA_PARAMS = {
    'n1': 'all',
    'p' : '202401-202410',
    'v' : '63'}

param_str:str = "/".join([f"{key}/{value}" for key, value in SIDRA_IPCA_PARAMS.items()])
url: str = f"{SIDRA_BASE_URL}/t/{SIDRA_IPCA_TABLEID}/{param_str}"

try:
  response: requests.Response = requests.get(url)
  response.raise_for_status()
  print(response.text)

except requests.RequestException as e:
  print(e)
  print(response.text)

In [ ]:
get_ipca_variacao_mensal(12)

Variação mensal do IPCA (Brasil) - Últimos 12 meses (via JSON):


,Mes,VariacaoMensal
0,outubro 2024,0.56
1,novembro 2024,0.39
2,dezembro 2024,0.52
3,janeiro 2025,0.16
4,fevereiro 2025,1.31
5,março 2025,0.56
6,abril 2025,0.43
7,maio 2025,0.26
8,junho 2025,0.24
9,julho 2025,0.26


### INPC - Índice Nacional de Preços ao Consumidor

O Índice Nacional de Preços ao Consumidor – INPC que tem por objetivo a correção do poder de compra dos salários, através da mensuração das variações de preços da cesta de consumo da população assalariada com mais baixo rendimento. Esta faixa de renda foi criada com o objetivo de garantir uma cobertura populacional de 50% das famílias cuja pessoa de referência é assalariada e pertencente às áreas urbanas de cobertura do SNIPC - Sistema Nacional de Índices de Preços ao Consumidor.

Esse índice de preços tem como unidade de coleta estabelecimentos comerciais e de prestação de serviços, concessionária de serviços públicos e internet e sua coleta estende-se, em geral, do dia 01 a 30 do mês de referência.

Atualmente, a população-objetivo do INPC abrange as famílias com rendimentos de 1 a 5 salários mínimos, cuja pessoa de referência é assalariada, residentes nas áreas urbanas das regiões de abrangência do SNIPC, as quais são: regiões metropolitanas de Belém, Fortaleza, Recife, Salvador, Belo Horizonte, Vitória, Rio de Janeiro, São Paulo, Curitiba, Porto Alegre, além do Distrito Federal e dos municípios de Goiânia, Campo Grande, Rio Branco, São Luís e Aracaju.